In [3]:
from pathlib import Path
import pandas as pd

RAW_DIR = Path("data/raw")
OUT_CSV = Path("data/completo.csv")

# ---------- 1. Helpers ----------
def leer_csv(path: Path, dtypes: dict = None):
    """Lee un CSV con control de tipos y sin dividir la memoria."""
    return pd.read_csv(path, dtype=dtypes or {}, low_memory=False)

def renombrar(df, mapping):
    """Renombra columnas si existen en el DF."""
    cols_presentes = {k: v for k, v in mapping.items() if k in df.columns}
    return df.rename(columns=cols_presentes)

# ---------- 2. Catálogos ----------
cat_banco = leer_csv(
    RAW_DIR / "CatBanco.csv",
    dtypes={"IdBanco": "Int64", "Nombre": "string"}
)
cat_banco = renombrar(cat_banco, {"Nombre": "NombreBanco"})

cat_respuesta = leer_csv(
    RAW_DIR / "CatRespuestaBancos.csv",
    dtypes={"IdRespuestaBanco": "string", "Descripcion": "string"}
)
cat_respuesta = renombrar(cat_respuesta, {"Descripcion": "DescripcionRespBanco"})

cat_emisora = leer_csv(
    RAW_DIR / "CatEmisora.csv",
    dtypes={
        "idEmisora": "Int64",
        "IdBanco": "Int64",
        "Nombre": "string",
        "TipoEnvio": "string",
        "Emisora": "string",
    },
)
cat_emisora = renombrar(cat_emisora, {"Nombre": "NombreEmisora"})

lista_cobro = leer_csv(
    RAW_DIR / "ListaCobro.csv",
    dtypes={
        "idListaCobro": "Int64",
        "idBanco": "Int64",
        "fechaCreacionLista": "string",
        "fechaEnvioCobro": "string",
    },
)

lista_cobro_emisora = leer_csv(
    RAW_DIR / "ListaCobroEmisora.csv",
    dtypes={"idListaCobro": "Int64", "idEmisora": "Int64"},
)

# ---------- 3. Detalles (union de todos los años) ----------
dtype_detalle = {
    "consecutivoCobro": "Int64",
    "fechaCobroBanco": "string",
    "idBanco": "Int64",
    "idCredito": "Int64",
    "idListaCobro": "Int64",
    "idRespuestaBanco": "string",   # ← clave para evitar floats
    "montoCobrado": "float64",
    "montoCobrar": "float64",
    "montoExigible": "float64",
}

df_detalles = []
for f in sorted(RAW_DIR.glob("ListaCobroDetalle*.csv")):
    print(f"📄 Leyendo {f.name}")
    df_detalles.append(leer_csv(f, dtype_detalle))

detalle = pd.concat(df_detalles, ignore_index=True)

# ---------- 4. JOINs controlados ----------
print("🔗 Join CatBanco")
detalle = (
    detalle.merge(cat_banco, how="left", left_on="idBanco", right_on="IdBanco")
    .drop(columns=["IdBanco"])
)

print("🔗 Join CatRespuestaBancos")
detalle = detalle.merge(cat_respuesta, how="left", left_on="idRespuestaBanco",
                        right_on="IdRespuestaBanco").drop(columns=["IdRespuestaBanco"])

print("🔗 Join ListaCobro")
detalle = detalle.merge(lista_cobro, how="left", on="idListaCobro",
                        suffixes=("", "_lista"))

print("🔗 Join ListaCobroEmisora")
detalle = detalle.merge(lista_cobro_emisora, how="left", on="idListaCobro")

print("🔗 Join CatEmisora")
detalle = detalle.merge(cat_emisora, how="left", on="idEmisora")

# ---------- 5. Guardar resultado ----------
detalle.to_csv(OUT_CSV, index=False)
print(f"\n✅ Archivo final generado en: {OUT_CSV.resolve()}")

📄 Leyendo ListaCobroDetalle2022.csv
📄 Leyendo ListaCobroDetalle2023.csv
📄 Leyendo ListaCobroDetalle2024.csv
📄 Leyendo ListaCobroDetalle2025.csv
🔗 Join CatBanco
🔗 Join CatRespuestaBancos
🔗 Join ListaCobro
🔗 Join ListaCobroEmisora
🔗 Join CatEmisora

✅ Archivo final generado en: /Users/maxzemeno/Documents/GitHub/credit-payment-optimization-model/data/completo.csv
